In [1]:
import requests
init = 'https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial'
response = requests.get(init)
response

<Response [200]>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
html = response.text
html

'<!DOCTYPE html>\n<html class="client-nojs" lang="pt" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Inteligência artificial – Wikipédia, a enciclopédia livre</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\\t.","\xa0\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janeiro","fevereiro","março","abril","maio","junho","julho","agosto","setembro","outubro","novembro","dezembro"],"wgRequestId":"9f707eda-420c-43df-afe8-b1f06dc38b56","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Inteligência_artificial","wgTitle":"Inteligência artificial","wgCurRevisionId":61440060,"wgRevisionId":61440060,"wgArticleId":12722,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["!Páginas com erros em referências","!CS1 manut: Nomes múltiplos: lista de autores","!CS1 inglês-fon

In [5]:
from bs4 import BeautifulSoup
bs = BeautifulSoup(html, 'html.parser')
#bs

In [6]:
links = bs('a')
links[0:2]

[<a class="mw-jump-link" href="#content">Saltar para o conteúdo</a>,
 <a class="mw-logo" href="/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal">
 <img alt="" aria-hidden="true" class="mw-logo-icon" height="50" src="/static/images/mobile/copyright/wikipedia.png" width="50"/>
 <span class="mw-logo-container">
 <img alt="Wikipédia" class="mw-logo-wordmark" height="18" src="/static/images/mobile/copyright/wikipedia-wordmark-fr.svg" width="119"/>
 <img alt="" class="mw-logo-tagline" height="13" src="/static/images/mobile/copyright/wikipedia-tagline-pt.svg" width="113"/>
 </span>
 </a>]

In [7]:
hrefs = [ l.get('href') for l in links]
hrefs[:10]

['#content',
 '/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal',
 '/wiki/Especial:Pesquisar',
 '/w/index.php?title=Especial:Criar_conta&returnto=Intelig%C3%AAncia+artificial',
 '/w/index.php?title=Especial:Criar_conta&returnto=Intelig%C3%AAncia+artificial',
 '/w/index.php?title=Especial:Entrar&returnto=Intelig%C3%AAncia+artificial',
 '/wiki/Ajuda:Introduction',
 '/wiki/Especial:Minha_discuss%C3%A3o',
 '/wiki/Especial:Minhas_contribui%C3%A7%C3%B5es',
 '/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal']

In [9]:
import os
from requests.compat import urljoin
import requests
from bs4 import BeautifulSoup
# funcionamento do urljoin
# https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial, /wiki/Alan_Turing
#  --> https://pt.wikipedia.org/wiki/Alan_Turing
# https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial, https://www.cell.com/action/captchaChallenge?redirectUri=%2Fcell%2Fpdf%2FS0092-8674%2818%2930154-5.pdf
#  --> https://www.cell.com/action/captchaChallenge?redirectUri=%2Fcell%2Fpdf%2FS0092-8674%2818%2930154-5.pdf
def salva_arquivo(pasta, nome, conteudo):  
  with open(f'{pasta}/{nome}.html', 'w', encoding='utf8') as file:
    file.write(conteudo)

from requests.compat import urljoin
def busca_links(conteudo, link, bs):
  #bs = BeautifulSoup(conteudo, 'html.parser')
  links = bs('a', href=links_validos)
  links = [ urljoin(link, l.get('href')) for l in links]
  return links

def links_validos(href):
  return href and href[0] != '#' and  href.startswith('/wiki/') # incluir regra para captura apenas links que tenham /wiki

def crawler(buscar, buscados, stop=1):
  if buscar and len(buscados) < stop:
    link = buscar.pop()
    if link not in buscados:
      response = requests.get(link)
      html = response.text
      bs = BeautifulSoup(html, 'html.parser')
      for remove in bs(['script','style']): remove.extract()
      titulo = bs('title')[0].text.replace('/','-')
      content = bs(id='content')[0].text
      salva_arquivo(
          f'/content/drive/MyDrive/Colab Notebooks/wiki',f'{titulo}', content
      )
      buscados.add(link)
      novos_links = busca_links(html, link, bs)
      buscar.update(novos_links)
    crawler(buscar, buscados, stop)
      

init = 'https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial'
crawler(set([init]),set([]),10)
